In [1]:
import pygame
from pygame.locals import *
import numpy.random as rnd

In [4]:
class Agent():
    
    def __init__(self,pos_init=(200,100),size=(20,20),maxstep=5,energy=255,spots_offset=5,existence_energy=0.05,num_obstacles=5):
        
        self.pos = pos_init
        self.energy = energy
        self.existence_energy = existence_energy
        self.maxstep = maxstep
        self.size = size
        self.spots_offset = spots_offset
        self.pain_t = 0
        self.pain_r = 0
        self.pain_l = 0
        self.pain_b = 0
        
        self.dir = [0,0]
        
        self.rect = pygame.rect.Rect((self.pos[0],self.pos[1],self.size[0],self.size[1]))
        
        self.rect_t = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1],self.size[0]-2*self.spots_offset,1))
        self.rect_r = pygame.rect.Rect((self.pos[0]+self.size[0]-1,self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_l = pygame.rect.Rect((self.pos[0],self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_b = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1]+self.size[1]-1,self.size[0]-2*self.spots_offset,1))
            
    def is_dead(self):
        if self.energy<10:
            return True
        return False
        
    def get_pos(self):
        return (self.rect[0],self.rect[1])
    
    def expend_existence_energy(self):
        self.energy -= self.existence_energy
        
    def accelerate(self,vvec):
        newdir = [self.dir[0]+vvec[0],self.dir[1]+vvec[1]]
        if newdir[0]<=self.maxstep and newdir[0]>=-self.maxstep:
            self.dir[0] = newdir[0]
        if newdir[1]<=self.maxstep and newdir[1]>=-self.maxstep:
            self.dir[1] = newdir[1]
        
    def move(self,env):
        screen_rect = env.screen.get_rect()
            
        self.pain_r = False
        self.pain_l = False
        self.pain_b = False
        if any([self.rect_t.colliderect(o) for o in env.obstacles]) or self.rect_t[1]==0:
            self.pain_t += 1
        else:
            if self.pain_t>0:
                self.pain_t -= 1
        if any([self.rect_r.colliderect(o) for o in env.obstacles]) or self.rect_r[0]+1==1200:
            self.pain_r += 1
        else:
            if self.pain_r>0:
                self.pain_r -= 1
        if any([self.rect_l.colliderect(o) for o in env.obstacles]) or self.rect_l[0]==0:
            self.pain_l += 1
        else:
            if self.pain_l>0:
                self.pain_l -= 1
        if any([self.rect_b.colliderect(o) for o in env.obstacles]) or self.rect_b[1]+1==600:
            self.pain_b += 1
        else:
            if self.pain_b>0:
                self.pain_b -= 1
        if any([self.rect.colliderect(o) for o in env.obstacles]) or self.collideswith_screenedge():
            self.energy -= 1
        
        self.decide_action()
        
        self.rect.move_ip(self.dir[0],0)
        self.rect.move_ip(0,self.dir[1])
        self.rect.clamp_ip(screen_rect)
        
        self.pos = (self.rect[0],self.rect[1]) 
        
        self.rect_t = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1],self.size[0]-2*self.spots_offset,1))
        self.rect_r = pygame.rect.Rect((self.pos[0]+self.size[0]-1,self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_l = pygame.rect.Rect((self.pos[0],self.pos[1]+self.spots_offset,1,self.size[1]-2*self.spots_offset))
        self.rect_b = pygame.rect.Rect((self.pos[0]+self.spots_offset,self.pos[1]+self.size[1]-1,self.size[0]-2*self.spots_offset,1))
   
    def collideswith_screenedge(self):
        position = self.get_pos()
        if position[0]==0 or position[0]==1200-self.size[0]:
            return True
        if position[1]==0 or position[1]==600-self.size[1]:
            return True
        
        return False

    def decide_action(self):
        
        if self.pain_t:
            self.accelerate([0,1])
        if self.pain_r:
            self.accelerate([-1,0])
        if self.pain_l:
            self.accelerate([1,0])
        if self.pain_b:
            self.accelerate([0,-1])
        
class Environment():
    
    def __init__(self,num_obstacles,num_greenbeans=1):
        self.screen = pygame.display.set_mode((1200,600))
        self.clock = pygame.time.Clock()
        self.has_greenbean = False
        
        self.obstacles = []
        for k in range(num_obstacles):
            crect = pygame.rect.Rect(rnd.randint(50,1100,1),rnd.randint(50,500,1),rnd.randint(20,100,1),rnd.randint(20,100,1))
            self.obstacles.append(crect)
            
        self.greenbeans = []
        for k in range(num_greenbeans):
            cgreen = Greenbean(pos=(400,400))
            self.greenbeans.append(cgreen)
            if self.has_greenbean==False:
                self.has_greenbean = True
        
    def draw_env(self):
        self.screen.fill(0)
        for obstacle in self.obstacles:
            pygame.draw.rect(self.screen,(0,255,0),obstacle)
        if self.has_greenbean:
            for bean in self.greenbeans:
                pygame.draw.rect(self.screen,(255,0,255),bean.beanrect)
        
    def draw_player(self,agent):
        if int(agent.energy)<=255: 
            brightness = int(agent.energy) 
        else: 
            brightness = 255
        pygame.draw.rect(self.screen,(0,0,brightness),agent.rect)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_t)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_r)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_l)
        pygame.draw.rect(self.screen,(255,255,255),agent.rect_b)

class Greenbean():
    
    def __init__(self,pos,size=(5,5),carried_energy=50):
        self.pos = pos
        self.size = size
        self.beanrect = pygame.rect.Rect(self.pos[0],self.pos[1],self.size[0],self.size[1])

In [5]:
num_obstacles = 5

pygame.init()
env = Environment(num_obstacles)
player = Agent()

try:
    
    running = True
    while running==True:
        
        env.draw_env()
        if not player.is_dead():
            player.expend_existence_energy()
            env.draw_player(player)
            player.move(env)
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type==KEYDOWN:
                if event.key==K_RIGHT:
                    player.accelerate([1,0])
                if event.key==K_LEFT:
                    player.accelerate([-1,0])
                if event.key==K_UP:
                    player.accelerate([0,-1])
                if event.key==K_DOWN:
                    player.accelerate([0,1])
                if event.key==K_ESCAPE:
                    running=False
                    pygame.quit()
        
        env.clock.tick(100)
        
except SystemExit:
    pygame.quit()